# Assignment 1 - Part of Speech Tagging

## Dependencies

In [1]:
# !pip install lightning
# !pip install torchtext.data
# !pip install torchtext
# !pip install torch

In [2]:
# TODO: remove unused dependencies

# file management
import urllib
from pathlib import Path
import zipfile
import os

# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from tqdm import tqdm

# pytorch
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# pytorch lightning
from lightning import LightningModule
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

# Glove and vocabulary
from torchtext.vocab import GloVe, build_vocab_from_iterator

## TASK 1: Corpus

* **Download** the corpus.
* **Encode** the corpus into a pandas.DataFrame object.
* **Split** it in training, validation, and test sets.

### Download the corpus

In [3]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)


def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")


def extract_dataset(download_path: Path, extract_path: Path):
    print("Extracting dataset... (it may take a while...)")
    with zipfile.ZipFile(download_path, 'r') as zip_file:
        zip_file.extractall(extract_path)

    print("Extraction completed!")

    Path.unlink(download_path)
    print("Deleted .zip dataset file")

In [4]:
url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dependency_treebank"

print(f"Current work directory: {Path.cwd()}")

dataset_folder = Path.cwd().joinpath("Datasets")

if not dataset_folder.exists():
    dataset_folder.mkdir(parents=True)

dataset_zip_path = dataset_folder.joinpath("dependency_treebank.zip")
dataset_path = dataset_folder.joinpath(dataset_name)

if not dataset_zip_path.exists():
    download_dataset(dataset_zip_path, url)

if not dataset_path.exists():
    extract_dataset(dataset_zip_path, dataset_folder)


Current work directory: c:\Users\merli\OneDrive\Desktop\Github repos\NLP\A1


### Encode the corpus into a pandas.DataFrame object and split it into train, validation and test sets

The corpus contains 200 documents.

   * **Train**: Documents 1-100
   * **Validation**: Documents 101-150
   * **Test**: Documents 151-199

In [5]:
dataframe_rows = []
id = 0

for i, file_path in enumerate(sorted(dataset_path.iterdir())):
    if file_path.is_file():  # split corpus documents in the tree categories: train, validation, tests
        if 1 <= i + 1 <= 100:
            split = 'train'
        elif 101 <= i + 1 <= 150:
            split = 'validation'
        else:
            split = 'test'

        with file_path.open(mode='r', encoding='utf-8') as text_file:  # read corpus lines
            lines = text_file.readlines()

        for line in lines:
            fields = line.strip().split('\t')
            if len(fields) == 1:
                id = id + 1
            if len(fields) >= 2:
                text = fields[0]  # store the first field as 'text'
                POS = fields[1]  # store the second field as 'POS'
                dataframe_row = {  #build DataFrame rows
                    "text": text,
                    "POS": POS,
                    "split": split,
                    "id": id
                }

                dataframe_rows.append(dataframe_row)  #append rows
# corpus DataFrame
corpus_df = pd.DataFrame(dataframe_rows)

#### Data inspection

In [6]:
corpus_df.head(10)

,text,POS,split,id
0,Pierre,NNP,train,0
1,Vinken,NNP,train,0
2,",",",",train,0
3,61,CD,train,0
4,years,NNS,train,0
5,old,JJ,train,0
6,",",",",train,0
7,will,MD,train,0
8,join,VB,train,0
9,the,DT,train,0


In [7]:
print("Dataframe structure:")
print(corpus_df)
print()

print(f"Total rows {len(corpus_df)}")
print()

Dataframe structure:
          text  POS  split    id
0       Pierre  NNP  train     0
1       Vinken  NNP  train     0
2            ,    ,  train     0
3           61   CD  train     0
4        years  NNS  train     0
...        ...  ...    ...   ...
94079  quarter   NN   test  3715
94080       of   IN   test  3715
94081     next   JJ   test  3715
94082     year   NN   test  3715
94083        .    .   test  3715

[94084 rows x 4 columns]

Total rows 94084



In [8]:
# Train, test, validation split
df_train = corpus_df[corpus_df['split'] == 'train'].drop(columns=['split'])
df_test = corpus_df[corpus_df['split'] == 'test'].drop(columns=['split'])
df_val = corpus_df[corpus_df['split'] == 'validation'].drop(columns=['split'])

## TASK 2: Text encoding

* Embed words using **GloVe embeddings**.
* TODO: see if we want to do it, otherwise remove it -> [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

In [9]:
def load_embedding_model(embedding_dimension: int = 300):
    emb_model = GloVe(name="6B", dim=embedding_dimension)
    return emb_model

In [10]:
punctuation_and_symbol_pos = [".", ",", ":", '``', "''", "$", "#", "-LRB-", "-RRB-", "SYM", "LS"] #TODO check "LS" 

### TASK 4.b: OOV tokens

Our vocabulary is stored in the GloVe object, and we simply edit its fields to add tokens and embedding vectors

In [11]:
# Find training set OOV tokens
embedding_dim = 50
embedder = load_embedding_model(embedding_dim)

existing_vocab_tokens = set(embedder.itos)  # Tokens in the vocabulary, i.e. present in GloVe
train_text = set([word.lower() for word in df_train['text']])

train_oov_tokens = train_text - existing_vocab_tokens
print(f"OOV tokens in the training set: {len(train_oov_tokens)}")
print(f"Some OOV tokens: {list(train_oov_tokens)[:50]}")

OOV tokens in the training set: 359
Some OOV tokens: ['savers\\/investors', '520-lawyer', 'glenham', '3\\/4', 'index-arbitrage', 'ghkm', '436.01', '37-a-share', 'new-home', 'wtd', 'sometimes-tawdry', 'anti-china', 'intellectual-property', 'one-country', '-lrb-', 'bridgestone\\/firestone', '84-month', 'purepac', 'boorse', 'school-research', '2645.90', 'integra-a', 'junk-bond', '1\\/4', 'equal-opportunity', 'landonne', '18,444', 'satrum', 'rope-sight', 'nekoosa', 'language-housekeeper', 'tarwhine', 'securities-based', 'purhasing', '446.62', 'ratners', 'abortion-related', '497.34', 'chilver', 'less-serious', '456.64', 'incentive-bonus', 'school-board', 'veselich', 'we-japanese', 'higher-salaried', 'hummerstone', 'test-prep', 'ariail', '1.457']


In [12]:
# Add training set OOV tokens to the GloVe embedder, sampling from a random uniform distribution for each feature in the respective range
mins = torch.min(embedder.vectors, dim=0).values
ranges = mins - torch.max(embedder.vectors, dim=0).values

for token in train_oov_tokens:
    embedder.itos.append(token)
    embedder.stoi[token] = len(embedder.itos) - 1
    embedder.vectors = torch.cat((embedder.vectors, (torch.rand(embedding_dim) * ranges + mins).unsqueeze(dim=0)), dim=0)

# For the '[UNK]' token embedding, sample a vector from a normal distribution for each feature

# Mean and std of the GloVe embeddings, for each feature
means = torch.mean(embedder.vectors, dim=0)
stds = torch.std(embedder.vectors, dim=0)
unk_vector = torch.normal(means, stds)

# The '[UNK]' token is not really added to the vocabulary (i.e. GloVe)
# instead, we redefine the 'unk_init' function of the embedder to return the embedding vector corresponding to '[UNK]'
embedder.unk_init = lambda x: unk_vector

print(f"New vocabulary size: {len(embedder.itos)}")

assert len(embedder.itos) == len(embedder.stoi) == embedder.vectors.shape[0]

New vocabulary size: 400359


### Embed words using GloVe embeddings

For the token to embedding mapping, we decided to use the following approach: the Dataset object takes care of calling the embedder every time it is queried to return a datapoint (\_\_getitem\_\_ method)

In [13]:
iterator = ([pos] for pos in corpus_df["POS"].unique())
pos_vocab = build_vocab_from_iterator(iterator)
pos_vocab.append_token("<PAD>")

pos_padding_value = pos_vocab["<PAD>"]
punctuation_and_symbol_pos_indices = [pos_vocab[token] for token in punctuation_and_symbol_pos]


class CorpusDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, embedder):
        min_id = dataframe['id'].min()
        dataframe['id'] = dataframe['id'] - min_id
        self.dataframe = dataframe.groupby("id")
        self.embedder = embedder

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sentence = self.dataframe.get_group(idx)
        text = sentence['text'].to_list()
        text = [token.lower() for token in text]        
        
        POS = sentence['POS'].to_list()
        POS = torch.Tensor([pos_vocab[token] for token in POS])

        embedded_text = self.embedder.get_vecs_by_tokens(text)

        return embedded_text, POS


In [14]:
# Definition of the dataset
dataset_train = CorpusDataset(df_train, embedder)
dataset_test = CorpusDataset(df_test, embedder)
dataset_val = CorpusDataset(df_val, embedder)
dataset_all = CorpusDataset(corpus_df, embedder)    # TODO: remove if not used


# This collate function takes care of adding padding to the sequences
# TODO - test if it works in the LSTM training
def my_collate(batch):
    sequences, labels = zip(*batch)

    # max_len = max([len(seq) for seq in sequences])
    # sequences_padded = [seq + ["<PAD>"] * (max_len - len(seq)) for seq in sequences]

    sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=0)
    labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=pos_padding_value)

    sequences_padded = sequences_padded.type(torch.float)
    labels_padded = labels_padded.type(torch.long)

    return [sequences_padded, labels_padded]


train_loader = DataLoader(dataset_train, batch_size=32, collate_fn=my_collate)
val_loader = DataLoader(dataset_val, batch_size=32, collate_fn=my_collate)
test_loader = DataLoader(dataset_test, batch_size=32, collate_fn=my_collate)

## TASK 4: Metrics

* Evaluate models using macro F1-score, computed over **all** tokens.

We defined our own macro F1-score metric in order to accumulate FP, TP, FN, TN iteratively

In [15]:
from torchmetrics import Metric
from torchmetrics import ConfusionMatrix


class F1ScoreCustom(Metric):
    def __init__(self, num_classes: int, pos_padding_value: int = pos_padding_value, punctuation_and_symbol_pos_indices: list = punctuation_and_symbol_pos_indices):
        super().__init__()

        self.num_classes = num_classes
        self.mask = torch.ones([num_classes])
        self.mask[[pos_padding_value] + punctuation_and_symbol_pos_indices] = 0
        
        self.add_state("true_positive", default=torch.zeros([num_classes]), dist_reduce_fx="sum")
        self.add_state("false_negative", default=torch.zeros([num_classes]), dist_reduce_fx="sum")
        self.add_state("false_positive", default=torch.zeros([num_classes]), dist_reduce_fx="sum")

    def update(self, y_hat_class: torch.Tensor, y_class: torch.Tensor):
        confusion_matrix_metric = ConfusionMatrix(num_classes=self.num_classes, task="multiclass")
        confusion_matrix = confusion_matrix_metric(y_hat_class, y_class)

        # # Confusion matrix, TP, FN and FP for class 0 
        # #   TRUE LABEL
        # #   0               TP     FN     FN     FN     FN       
        # #   1               FP       
        # #   2               FP               
        # #   3               FP                       
        # #   4               FP                                
        # # PREDICTED LABEL   0       1      2      3      4
        # 

        true_positive = torch.Tensor([confusion_matrix[i][i] for i in range(self.num_classes)])
        false_negative = torch.Tensor([sum(confusion_matrix[i, :]) - true_positive[i] for i in range(self.num_classes)])
        false_positive = torch.Tensor([sum(confusion_matrix[:, i]) - true_positive[i] for i in range(self.num_classes)])

        self.true_positive += true_positive
        self.false_negative += false_negative
        self.false_positive += false_positive

    def compute(self):
        precision = self.true_positive / (self.true_positive + self.false_positive)
        recall = self.true_positive / (self.true_positive + self.false_negative)

        f1 = 2 * (precision * recall) / (precision + recall)

        f1 = f1 * self.mask

        return f1



## TASK 3: Model definition

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

### Baseline model: Bidirectional LSTM + Dense layer

In [16]:
class BiLSTMModel(LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, target_padding_value=pos_padding_value):
        super(BiLSTMModel, self).__init__()
        self.output_dim = output_dim
        self.target_padding_value = target_padding_value

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            batch_first=True,
                            bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplied by 2 due to the bidirectionality

        self._train_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._val_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._test_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)

    def forward(self, x):
        # embedding = self.embedding_layer(x)
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc(lstm_out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        # Change shape from (batchsize, sequence_len, classes) to be (batchsize, classes, sequence_len) to compute loss function
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'train_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      on_step=False, reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._train_f1_metric.update(y_hat_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)

        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)
        self.log_dict({'val_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._val_f1_metric.update(y_hat_class, y)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'test_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._test_f1_metric.update(y_hat_class, y)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _compute_f1(self, f1_metric):
        mean_f1_score = f1_metric.compute()

        # Create a mask that is False for NaNs
        mask = torch.isnan(mean_f1_score)

        # Invert the mask: True for valid entries, False for NaNs
        valid_data = mean_f1_score[~mask]

        # Compute the mean of the non-NaN values
        mean_value = torch.mean(valid_data)

        return mean_value

    def on_train_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._train_f1_metric)
        self.log_dict({"train_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._train_f1_metric.reset()

    def on_validation_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._val_f1_metric)
        self.log_dict({"val_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._val_f1_metric.reset()

    def on_test_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._test_f1_metric)
        self.log_dict({"test_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._test_f1_metric.reset()


### Model 1: Bidirectional 2-layers LSTM + Dense layer

In [17]:
class Model1(LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, target_padding_value=pos_padding_value):
        super(Model1, self).__init__()
        self.output_dim = output_dim
        self.target_padding_value = target_padding_value

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplied by 2 due to the bidirectionality

        self._train_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._val_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._test_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)

    def forward(self, x):
        # embedding = self.embedding_layer(x)
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc(lstm_out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        # Change shape from (batchsize, sequence_len, classes) to be (batchsize, classes, sequence_len) to compute loss function
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'train_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      on_step=False, reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._train_f1_metric.update(y_hat_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)

        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)
        self.log_dict({'val_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._val_f1_metric.update(y_hat_class, y)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'test_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._test_f1_metric.update(y_hat_class, y)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _compute_f1(self, f1_metric):
        mean_f1_score = f1_metric.compute()

        # Create a mask that is False for NaNs
        mask = torch.isnan(mean_f1_score)

        # Invert the mask: True for valid entries, False for NaNs
        valid_data = mean_f1_score[~mask]

        # Compute the mean of the non-NaN values
        mean_value = torch.mean(valid_data)

        return mean_value

    def on_train_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._train_f1_metric)
        self.log_dict({"train_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._train_f1_metric.reset()

    def on_validation_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._val_f1_metric)
        self.log_dict({"val_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._val_f1_metric.reset()

    def on_test_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._test_f1_metric)
        self.log_dict({"test_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._test_f1_metric.reset()

### Model 2: Bidirectional LSTM + 2 Dense layers

In [18]:
class Model2(LightningModule):
    def __init__(self, input_dim, hidden_dim, output_dim, fc_size,
                 target_padding_value=pos_padding_value):
        super(Model2, self).__init__()
        self.output_dim = output_dim
        self.target_padding_value = target_padding_value

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.fc_1 = nn.Linear(hidden_dim * 2, fc_size)  # Multiplied by 2 due to bidirectionality
        self.fc_2 = nn.Linear(fc_size, output_dim)

        self._train_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._val_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)
        self._test_f1_metric = F1ScoreCustom(num_classes=output_dim, pos_padding_value=self.target_padding_value)

    def forward(self, x):
        # embedding = self.embedding_layer(x)
        lstm_out, _ = self.lstm(x)
        # lstm_out (batch_size, seq_length, hidden_size * 2)
        out = self.fc_1(lstm_out)
        out = self.fc_2(out)
        # out (batch_size, seq_length, output_dim)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        # Change shape from (batchsize, sequence_len, classes) to be (batchsize, classes, sequence_len) to compute loss function
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'train_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      on_step=False, reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._train_f1_metric.update(y_hat_class, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)

        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)
        self.log_dict({'val_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._val_f1_metric.update(y_hat_class, y)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat = torch.movedim(y_hat, 1, 2)
        loss = nn.functional.cross_entropy(y_hat, y, ignore_index=self.target_padding_value)

        self.log_dict({'test_loss': loss, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True,
                      reduce_fx="mean")

        y_hat_class = torch.argmax(y_hat, dim=1)
        self._test_f1_metric.update(y_hat_class, y)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def _compute_f1(self, f1_metric):
        mean_f1_score = f1_metric.compute()

        # Create a mask that is False for NaNs
        mask = torch.isnan(mean_f1_score)

        # Invert the mask: True for valid entries, False for NaNs
        valid_data = mean_f1_score[~mask]

        # Compute the mean of the non-NaN values
        mean_value = torch.mean(valid_data)

        return mean_value

    def on_train_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._train_f1_metric)
        self.log_dict({"train_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._train_f1_metric.reset()

    def on_validation_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._val_f1_metric)
        self.log_dict({"val_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._val_f1_metric.reset()

    def on_test_epoch_end(self) -> None:
        mean_f1_score = self._compute_f1(self._test_f1_metric)
        self.log_dict({"test_f1": mean_f1_score, 'step': float(self.current_epoch)}, on_epoch=True, prog_bar=True, logger=True)
        self._test_f1_metric.reset()

## TASK 5: Training and Evaluation

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.

In [19]:
# Fix all possible sources of randomness
torch.use_deterministic_algorithms(True)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

### Training and evaluation

#### Training all models on training set and evalutating on validation set at every epoch s.t. to keep the best model for each model/seed pair.
#### Both cross entropy loss and F1-Score are computed at each evaluation

In [20]:
logs_path = Path.cwd() / "logs" / "lightning_logs"


seeds = [6, 90, 157]

epochs = 100
output_dim = len(df_train["POS"].unique()) + 1  # +1 for padding
hidden_dim = 128
input_dim = embedding_dim

model_classes = [BiLSTMModel, Model1, Model2]
model_names = ["baseline", "model1", "model2"]
hyperparameters = [
    {'input_dim': input_dim, 'hidden_dim': hidden_dim, 'output_dim': output_dim},
    {'input_dim': input_dim, 'hidden_dim': hidden_dim, 'output_dim': output_dim},
    {'input_dim': input_dim, 'hidden_dim': hidden_dim, 'output_dim': output_dim, 'fc_size': 64} 
]

for model_class, model_name, hyperparameter in zip(model_classes, model_names, hyperparameters):
    for seed in seeds:
        print(f"Training model {model_name} with seed {seed}...")
        seed_everything(seed, workers=True)

        model = model_class(**hyperparameter)

        logger = TensorBoardLogger(logs_path, name=f"{model_name}_seed{seed}")
        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=None,
            filename=f'{model_name}-seed={seed}' + '-{epoch:02d}-{val_loss:.2f}-{val_f1:.2f}',
            save_top_k=1,
        )
        early_stop_callback = EarlyStopping(
            monitor='val_loss',
            patience=3,
            verbose=True,
            mode='min'
        )


        trainer = Trainer(
            max_epochs=epochs,
            logger=logger,
            log_every_n_steps=1,
            callbacks=[checkpoint_callback, early_stop_callback],
            deterministic=True
        )
        
        trainer.fit(model, train_loader, val_loader)


Seed set to 6


Training model baseline with seed 6...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\merli\OneDrive\Desktop\Github repos\NLP\A1\logs\lightning_logs\baseline_seed6

  | Name             | Type          | Params
---------------------------------------------------
0 | lstm             | LSTM          | 184 K 
1 | fc               | Linear        | 11.8 K
2 | _train_f1_metric | F1ScoreCustom | 0     
3 | _val_f1_metric   | F1ScoreCustom | 0     
4 | _test_f1_metric  | F1ScoreCustom | 0     
---------------------------------------------------
196 K     Trainable params
0         Non-trainable params
196 K     Total params
0.785     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

d:\Users\edo\envs\nlp\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


...

Epoch 17: 100%|██████████| 59/59 [00:35<00:00,  1.65it/s, v_num=0, val_loss=0.444, step=17.00, val_f1=0.593, train_loss=0.181, train_f1=0.645]

Metric val_loss improved by 0.012 >= min_delta = 0.0. New best score: 0.444


Epoch 20: 100%|██████████| 59/59 [00:42<00:00,  1.38it/s, v_num=0, val_loss=0.504, step=20.00, val_f1=0.595, train_loss=0.127, train_f1=0.659]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.444. Signaling Trainer to stop.


Epoch 20: 100%|██████████| 59/59 [00:42<00:00,  1.38it/s, v_num=0, val_loss=0.504, step=20.00, val_f1=0.595, train_loss=0.127, train_f1=0.659]


### Robust estimation across seeds and choice of best model

In [ ]:
# test the model
# validation_results_baseline_model = trainer.validate(model, dataloaders=val_loader)

In [1]:
%reload_ext tensorboard

In [2]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 8392), started 3 days, 0:53:09 ago. (Use '!kill 8392' to kill it.)

## TASK 6: Error analysis

* Compare the errors made on the validation and test sets.
* Aggregate model errors into categories (if possible) 
* Comment about errors and propose possible solutions on how to address them.

# Task 7: Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.